In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor

# K-fold cross validation
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

import time
# to save the models
import pickle

# to remove unnecessary warnings
import warnings

warnings.filterwarnings('ignore')

# accuracy metrics for the regression problem
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from ipynb.fs.full.ML_algorithms import linear_regressor
from ipynb.fs.full.ML_algorithms import decision_tree_regressor
from ipynb.fs.full.ML_algorithms import random_forest_regressor
from ipynb.fs.full.ML_algorithms import gradient_boosting_regressor
# from ipynb.fs.full.ML_algorithms import X_gradient_boosting_regressor

import statsmodels.api as sm

#### Reading the dataset and one-hot encoding

In [57]:
# read the csv file in a dataframe
df = pd.read_csv("bmw_used_cars.csv")

# As I am using the 'car_age' feature, we don't need the redundant 'year' as a feature
df = df.drop('year', axis=1)

# Let's check the data
display(df.head())
display(df.info())

,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,Engine_Size,road_tax_range,mileage_range,car_age,car_age_range
0,5 Series,11200,Automatic,67068,Diesel,125,57.6,2.0,medium,below_150,Good,7,5<age<=10
1,6 Series,27000,Automatic,14827,Petrol,145,42.8,2.0,medium,below_150,Excellent,3,<=5
2,5 Series,16000,Automatic,62794,Diesel,160,51.4,3.0,large,150_300,Good,5,<=5
3,1 Series,12750,Automatic,26676,Diesel,145,72.4,1.5,medium,below_150,Excellent,4,<=5
4,7 Series,14500,Automatic,39554,Diesel,160,50.4,3.0,large,150_300,Excellent,7,5<age<=10


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10781 entries, 0 to 10780
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   model           10781 non-null  object 
 1   price           10781 non-null  int64  
 2   transmission    10781 non-null  object 
 3   mileage         10781 non-null  int64  
 4   fuelType        10781 non-null  object 
 5   tax             10781 non-null  int64  
 6   mpg             10781 non-null  float64
 7   engineSize      10781 non-null  float64
 8   Engine_Size     10734 non-null  object 
 9   road_tax_range  10441 non-null  object 
 10  mileage_range   10781 non-null  object 
 11  car_age         10781 non-null  int64  
 12  car_age_range   10781 non-null  object 
dtypes: float64(2), int64(4), object(7)
memory usage: 1.1+ MB


None

There are 12 features.
<br>Among them, there are 5 numerical features 'mileage', 'tax', 'mpg', 'engineSize', 'car_age'
<br>As 'mpg' and 'mileage' have few outliers, we'll standerdize these variables

#### Standardization

In [58]:
# numeric features
num_cols = ['mileage', 'tax', 'mpg', 'engineSize', 'car_age']

# instantiate the scaler
scale = StandardScaler()

# using a loop to standardize only the numeric columns
for col in num_cols:
    df[col] = scale.fit_transform(df[[col]])

display(df.head())

,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,Engine_Size,road_tax_range,mileage_range,car_age,car_age_range
0,5 Series,11200,Automatic,1.653447,Diesel,-0.108963,0.038326,-0.303911,medium,below_150,Good,1.310782,5<age<=10
1,6 Series,27000,Automatic,-0.424388,Petrol,0.216199,-0.433982,-0.303911,medium,below_150,Excellent,-0.392121,<=5
2,5 Series,16000,Automatic,1.483453,Diesel,0.460070,-0.159533,1.507591,large,150_300,Good,0.459331,<=5
3,1 Series,12750,Automatic,0.046894,Diesel,0.216199,0.510634,-1.209662,medium,below_150,Excellent,0.033605,<=5
4,7 Series,14500,Automatic,0.559104,Diesel,0.460070,-0.191445,1.507591,large,150_300,Excellent,1.310782,5<age<=10


#### one-hot encoding

In [59]:
# one-hot encoding
df = pd.get_dummies(df)

# Let's check the data
display(df.head())
display(df.info())

,price,mileage,tax,mpg,engineSize,car_age,model_ 1 Series,model_ 2 Series,model_ 3 Series,model_ 4 Series,...,road_tax_range_below_150,mileage_range_Bad,mileage_range_Excellent,mileage_range_Good,mileage_range_Medium,car_age_range_10<age<=15,car_age_range_15<age<=20,car_age_range_20<age<=25,car_age_range_5<age<=10,car_age_range_<=5
0,11200,1.653447,-0.108963,0.038326,-0.303911,1.310782,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
1,27000,-0.424388,0.216199,-0.433982,-0.303911,-0.392121,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
2,16000,1.483453,0.460070,-0.159533,1.507591,0.459331,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,12750,0.046894,0.216199,0.510634,-1.209662,0.033605,1,0,0,0,...,1,0,1,0,0,0,0,0,0,1
4,14500,0.559104,0.460070,-0.191445,1.507591,1.310782,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10781 entries, 0 to 10780
Data columns (total 55 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   price                     10781 non-null  int64  
 1   mileage                   10781 non-null  float64
 2   tax                       10781 non-null  float64
 3   mpg                       10781 non-null  float64
 4   engineSize                10781 non-null  float64
 5   car_age                   10781 non-null  float64
 6   model_ 1 Series           10781 non-null  uint8  
 7   model_ 2 Series           10781 non-null  uint8  
 8   model_ 3 Series           10781 non-null  uint8  
 9   model_ 4 Series           10781 non-null  uint8  
 10  model_ 5 Series           10781 non-null  uint8  
 11  model_ 6 Series           10781 non-null  uint8  
 12  model_ 7 Series           10781 non-null  uint8  
 13  model_ 8 Series           10781 non-null  uint8  
 14  model_

None

Before one-hot encoding, there were 12 features. 
<br>After one-hot encoding, there are 54 features.

### Feature Selection

Removing the features having variance less than 0.01

In [60]:
def variance_threshold_selector(data, threshold):
    """
    This function removes the features with a variance lower than the threshold
    Args:
        data (dataFrame): pandas DataFrame
        threshold (float): Features with variance lower than this threshold will be removed
    Returns:
        a dataFrame with the actual index and column names
    """
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    
    return data[data.columns[selector.get_support(indices=True)]]

In [61]:
df = variance_threshold_selector(df, 0.01)

display(df.head())
# display(df.info())

,price,mileage,tax,mpg,engineSize,car_age,model_ 1 Series,model_ 2 Series,model_ 3 Series,model_ 4 Series,...,Engine_Size_large,Engine_Size_medium,road_tax_range_150_300,road_tax_range_below_150,mileage_range_Excellent,mileage_range_Good,mileage_range_Medium,car_age_range_10<age<=15,car_age_range_5<age<=10,car_age_range_<=5
0,11200,1.653447,-0.108963,0.038326,-0.303911,1.310782,0,0,0,0,...,0,1,0,1,0,1,0,0,1,0
1,27000,-0.424388,0.216199,-0.433982,-0.303911,-0.392121,0,0,0,0,...,0,1,0,1,1,0,0,0,0,1
2,16000,1.483453,0.460070,-0.159533,1.507591,0.459331,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
3,12750,0.046894,0.216199,0.510634,-1.209662,0.033605,1,0,0,0,...,0,1,0,1,1,0,0,0,0,1
4,14500,0.559104,0.460070,-0.191445,1.507591,1.310782,0,0,0,0,...,1,0,1,0,1,0,0,0,1,0


After removing the features having variance less than 0.01, now we have 32 features
<br>That means, we have reduced the system complexity by [(54-32)/54]*100 = 41%

#### Splitting the data for training and testing

In [62]:
train = df.drop('price', axis=1)
target = df[['price']]

# training size = 80%, test size = 20%
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

print('X_train :', X_train.shape)
print('y_train :', y_train.shape)
print('X_test :', X_test.shape)
print('y_test :', y_test.shape)

X_train : (8624, 32)
y_train : (8624, 1)
X_test : (2157, 32)
y_test : (2157, 1)


#### Linear Regression

In [63]:
# using statsmodel for the training
model_LR = sm.OLS(y_train, X_train)
result_LR = model_LR.fit()
print(result_LR.summary())

# prediction
y_pred_LR = result_LR.predict(X_test)

# mean absolute error
MAE = mean_absolute_error(y_test, y_pred_LR)

# root mean squared error
RMSE = np.sqrt(mean_squared_error(y_test, y_pred_LR))

# coefficient of determination
r2 = r2_score(y_test, y_pred_LR)

print("MAE : ", MAE, "RMSE : ", RMSE, "r2_score :", r2)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     1416.
Date:                Sun, 04 Apr 2021   Prob (F-statistic):               0.00
Time:                        11:08:13   Log-Likelihood:                -85013.
No. Observations:                8624   AIC:                         1.701e+05
Df Residuals:                    8592   BIC:                         1.703e+05
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
mileage                 

Null hypothesis: There is no relationship between the target and the predictor variable
<br>When p-value < 0.05 for a feature, we can reject the null hypothesis, that means that feature has a relation with the target variable.
    
When p-value > 0.05, for the one-hot encoded features 'model_M4', 'model_X5', 'road_tax_range_150_300', we can't reject the null hypothesis. These features are not well-suited to fit the model using Linear Regression.

<br>MAE lower is better
<br>RMSE lower is better
<br> r2_score greater is better
<br>MAE and RMSE are very high for the Linear Regression model and r2_score is also low. So, we'll now move to ensemble learning.

#### Feature Selection method: Recursive Feature Elimination (RFE)¶
RFE is an efficient approach for eliminating features from a training dataset and it selects those features in a training dataset that are most relevant in predicting the target variable. There are two important configuration options when using RFE: the choice in the number of features to select (n_features_to_select) and the choice of the algorithm used to help choose features (estimator).

RFE works by searching for a subset of features by starting with all features in the training dataset and successfully removing features until the desired number remains. This is achieved by fitting the given machine learning algorithm used in the core of the model, ranking features by importance, discarding the least important features, and re-fitting the model. This process is repeated until a specified number of features remains.

I have used Gradient Boosting Regressor and done some experiment by varying the n_features_to_select from 101 to 50. Finally, I found that, starting with 101 features and removing 5 least important features at each iteration, up to n_features_to_select=60, the model performance remain the same.

ref: https://machinelearningmastery.com/rfe-feature-selection-in-python/

In [64]:
rfe = RFE(estimator=GradientBoostingRegressor(random_state=42), n_features_to_select=25, step=1, verbose=1)
rfe.fit(X_train, y_train)

Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.


RFE(estimator=GradientBoostingRegressor(random_state=42),
    n_features_to_select=25, verbose=1)

In [65]:
n_features = df.shape[1]

# list of selected features after the recursive feature elimination (RFE)
rfe_cols = X_train.columns[rfe.support_]

n_features_rfe = rfe_cols.shape[0]
print("Number of remaining features after Variance Threshold method and RFE = ", n_features_rfe)
print("Finally selected features for training = ", rfe_cols)

# percentage reduction of features from the actual number of features
p_reduced_features_rfe = int((n_features - n_features_rfe)*100/n_features)
print("Number of features reduced after Variance Thresholding and RFE = ", n_features - n_features_rfe)
print("Percentage of total features reduced = ", p_reduced_features_rfe, "%")

Number of remaining features after Variance Threshold method and RFE =  25
Finally selected features for training =  Index(['mileage', 'tax', 'mpg', 'engineSize', 'car_age', 'model_ 1 Series',
       'model_ 2 Series', 'model_ 3 Series', 'model_ 4 Series',
       'model_ 5 Series', 'model_ M4', 'model_ X1', 'model_ X3', 'model_ X4',
       'model_ X5', 'transmission_Automatic', 'transmission_Manual',
       'transmission_Semi-Auto', 'fuelType_Diesel', 'fuelType_Hybrid',
       'fuelType_Petrol', 'Engine_Size_large', 'Engine_Size_medium',
       'car_age_range_5<age<=10', 'car_age_range_<=5'],
      dtype='object')
Number of features reduced after Variance Thresholding and RFE =  8
Percentage of total features reduced =  24 %


In [66]:
# keeping on the selected features from lower variance features removal and RFE
# for cross-validation
X_train = X_train[rfe_cols]
X_test = X_test[rfe_cols]

print(X_train.shape, X_test.shape)

(8624, 25) (2157, 25)


In [67]:
model_LR = linear_regressor(X_train, y_train, [True], [True], 'neg_root_mean_squared_error', 5)

time =  0.001705332597096761


#### Decision Tree

In [68]:
model_DT = decision_tree_regressor(X_train, y_train, 
                                   ['mse'], 
                                   ['auto'], 
                                   np.arange(10,11,1).tolist(),
                                   np.arange(3,4,1).tolist(), 
                                   'neg_root_mean_squared_error', 
                                   5)

time =  0.0025554855664571127


#### Random Forest

In [69]:
model_RF = random_forest_regressor(X_train, y_train, 
                                   ['mse'], 
                                   np.arange(200,201,100).tolist(), 
                                   [False], 
                                   ['auto'],
                                   np.arange(10,11,1).tolist(), 
                                   np.arange(3,4,1).tolist(), 
                                   'neg_root_mean_squared_error', 
                                   5)

time =  0.2478763461112976


#### Gradient Boosting

In [70]:
model_GB = gradient_boosting_regressor(X_train, y_train, 
                                       ['mse'],
                                       np.arange(4,5,1).tolist(), 
                                       np.arange(100,101,100).tolist(), 
                                       [0.1], 
                                       'neg_root_mean_squared_error', 
                                       5)

time =  0.05970333814620972


#### Extreme Gradient Boosting

In [71]:
# model_XGB = X_gradient_boosting_regressor(X_train, y_train,
#                                           [3,4,5], 
#                                           [100,200,300], 
#                                           [0.1],
#                                           'neg_mean_squared_error', 5)

#### Saving the Cross-validation results

In [72]:
def cv_result(model):
    """
    This function returns the grid search and cross-validation results in a DataFrame
    Args:
        model: trained model
    Returns:
        a datafrmae
    """

    # create a new dataframe from the grid search cross-validation results
    cross_val_result = pd.DataFrame(model.cv_results_)
    
    # list of columns to drop
    drop_cols = ['mean_fit_time', 'std_fit_time', 'mean_score_time', 
                 'std_score_time', 'params']

    cross_val_result = cross_val_result.drop(drop_cols, axis=1)
    cross_val_result = cross_val_result.sort_values(by='rank_test_score')
    
    # return a dataframe
    return cross_val_result

#### Prediction and Key Performance Indicators

In [73]:
def model_output(X_test, y_test, model):
    
    print(model.best_params_)
    
    cv = cv_result(model)
    y_pred = model.predict(X_test)
    MAE = mean_absolute_error(y_test, y_pred)
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    display(cv.head())
    print("MAE : ", MAE, "RMSE : ", RMSE, "r2_score :", r2)
    
    return cv, y_pred, MAE, RMSE, r2

In [74]:
cv_DT, y_pred_DT, MAE_DT, RMSE_DT, r2_DT = model_output(X_test, y_test, model_DT)
cv_RF, y_pred_RF, MAE_RF, RMSE_RF, r2_RF = model_output(X_test, y_test, model_RF)
cv_GB, y_pred_GB, MAE_GB, RMSE_GB, r2_GB = model_output(X_test, y_test, model_GB)
cv_LR, y_pred_LR, MAE_LR, RMSE_LR, r2_LR = model_output(X_test, y_test, model_LR)
# cv_XGB, y_pred_XGB, MAE_XGB, RMSE_XGB, r2_XGB = model_output(X_test, y_test, model_XGB)

{'criterion': 'mse', 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 3}


,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,mse,10,auto,3,-2883.665766,-3373.263684,-3245.471112,-3653.872079,-4231.184216,-3477.491371,450.885511,1


MAE :  2183.504727118648 RMSE :  3482.056982037003 r2_score : 0.90623887085475
{'bootstrap': False, 'criterion': 'mse', 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 200}


,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_n_estimators,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,False,mse,10,auto,3,200,-2864.636009,-3178.314449,-3216.456454,-3511.325729,-4231.488545,-3400.444237,463.29536,1


MAE :  2183.9899663166907 RMSE :  3479.8334862486195 r2_score : 0.9063585764852726
{'criterion': 'mse', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}


,param_criterion,param_learning_rate,param_max_depth,param_n_estimators,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,mse,0.1,4,100,-2429.077052,-2797.126737,-2799.548915,-3042.902454,-3982.898503,-3010.310732,524.345257,1


MAE :  1872.3989092430934 RMSE :  3094.7182301380526 r2_score : 0.925938362916052
{'fit_intercept': True, 'normalize': True}


,param_fit_intercept,param_normalize,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,True,True,-4704.877118,-4828.914378,-4690.985974,-5152.461979,-5649.409373,-5005.329764,362.406576,1


MAE :  3227.02013021864 RMSE :  5009.771144763285 r2_score : 0.8059172866821752


In [75]:
y_test['DT'] = y_pred_DT
y_test['RF'] = y_pred_RF
y_test['GB'] = y_pred_GB
y_test['LR'] = y_pred_LR
# y_test['XGB'] = y_pred_XGB

y_test

,price,DT,RF,GB,LR
8728,15300,14895.600000,14895.600000,14439.149579,15589.195355
761,15495,13004.151685,13004.151685,13732.694886,12973.225451
7209,39875,38995.000000,38995.000000,43954.957038,40476.629182
6685,21730,21552.772727,21552.772727,20631.205872,22701.387638
8548,13799,17852.115385,17852.115385,15394.361033,17224.295449
...,...,...,...,...,...
10677,12000,12402.000000,12402.000000,12655.190237,11472.535286
8418,11759,10287.472727,10287.472727,10051.654222,9254.648516
1702,21460,23591.625000,23591.625000,22512.001276,22184.742897
6965,52991,50377.000000,50377.000000,48645.692926,48252.942011


In [76]:
# OUT = pd.DataFrame({'actual':np.squeeze(y_test.values), 
#                     'DT':y_pred_DT, 
#                     'RF':y_pred_RF, 
#                     'GB':y_pred_GB,
#                     'LR':np.squeeze(y_pred_LR),
#                     'XGB':y_pred_XGB})
# OUT = OUT.round(2)
# display(OUT)

In [77]:
# # to generate clear images
# sns.set_context('talk')

# # set the background of the images
# sns.set_style('darkgrid')

In [78]:
# plt.figure(figsize=(30,50))

# plt.subplot(7,1,1)
# plt.plot(OUT.index, OUT.actual-OUT.DT)

# plt.subplot(7,1,2)
# plt.plot(OUT.index, OUT.actual-OUT.RF, color='red')

# plt.subplot(7,1,3)
# plt.plot(OUT.index, OUT.actual-OUT.GB, color='green')
# plt.axhline(y=0, color='r', linestyle='-')
# plt.axhline(y=2000, color='r', linestyle='-')
# plt.axhline(y=-2000, color='r', linestyle='-')

# plt.subplot(7,1,4)
# plt.plot(OUT.index, OUT.actual-OUT.XGB, color='orange')

# plt.subplot(7,1,5)
# plt.plot(OUT.index, OUT.actual-OUT.LR, color='red', alpha=0.5)

# plt.subplot(7,1,6)
# plt.plot(OUT.index, OUT.actual-OUT.AB, color='green', alpha=0.5)

# plt.subplot(7,1,7)
# plt.plot(OUT.index, OUT.actual-OUT.EN, color='blue', alpha=0.5)
# # plt.plot(OUT.index, OUT.GB)